In [70]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [71]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = train[train.Utilities != 'NoSeWa']

train=train.drop(columns=['Id','Utilities', 'Street'])
test=test.drop(columns=['Id','Utilities', 'Street'])
train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Alley,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,NaN,Reg,Lvl,Inside,Gtl,CollgCr,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,NaN,Reg,Lvl,FR2,Gtl,Veenker,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,NaN,IR1,Lvl,Inside,Gtl,CollgCr,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,NaN,IR1,Lvl,Corner,Gtl,Crawfor,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,NaN,IR1,Lvl,FR2,Gtl,NoRidge,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [72]:
#null values in MSZoning test set. Drop C
train.loc[(train['MSZoning'] == 'C (all)') , 'MSZoning'] = np.nan
test.loc[(test['MSZoning'] == 'C (all)') , 'MSZoning'] = np.nan
train['MSZoning'] = train['MSZoning'].fillna(train['MSZoning'].mode()[0])
test['MSZoning'] = test['MSZoning'].fillna(train['MSZoning'].mode()[0])

#combine RH and RM into 1 category
train.loc[(train['MSZoning'] == 'RH') | (train['MSZoning'] == 'RM') , 'MSZoning'] = 'RHM'
test.loc[(test['MSZoning'] == 'RH') | (test['MSZoning'] == 'RM') , 'MSZoning'] = 'RHM'

#encode
train = pd.concat([train, pd.get_dummies(train['MSZoning'], drop_first=True, prefix='MSZoning')], axis = 1)
train=train.drop(columns=['MSZoning'])
test = pd.concat([test, pd.get_dummies(test['MSZoning'], drop_first=True, prefix='MSZoning')], axis = 1)
test=test.drop(columns=['MSZoning'])

In [73]:
#Lot Frontage
#null values
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
train['LotFrontage'] = imputer.fit_transform(train['LotFrontage'].values.reshape(-1, 1))
test['LotFrontage'] = imputer.transform(test['LotFrontage'].values.reshape(-1, 1))

# bands
# train['LotFrontage'] = pd.cut(train['LotFrontage'], 3)
train.loc[ train['LotFrontage'] <= 60, 'LotFrontage'] = 0
train.loc[(train['LotFrontage'] > 60) & (train['LotFrontage'] <= 100), 'LotFrontage'] = 1
train.loc[(train['LotFrontage'] > 100) & (train['LotFrontage'] <= 120), 'LotFrontage'] = 2
train.loc[(train['LotFrontage'] > 120), 'LotFrontage'] = 3

test.loc[ test['LotFrontage'] <= 60, 'LotFrontage'] = 0
test.loc[(test['LotFrontage'] > 60) & (test['LotFrontage'] <= 100), 'LotFrontage'] = 1
test.loc[(test['LotFrontage'] > 100) & (test['LotFrontage'] <= 120), 'LotFrontage'] = 2
test.loc[(test['LotFrontage'] > 120), 'LotFrontage'] = 3


In [74]:
#LotArea. non null
#scaled
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train['LotArea'] = sc.fit_transform(train['LotArea'].values.reshape(-1, 1))
test['LotArea'] = sc.transform(test['LotArea'].values.reshape(-1, 1))

In [75]:
#Alley
train.loc[(train['Alley'] == 'Pave') | (train['Alley'] == 'Grvl') , 'Alley'] = 'alley'
test.loc[(test['Alley'] == 'Pave') | (test['Alley'] == 'Grvl') , 'Alley'] = 'alley'
train['Alley'].fillna('No', inplace=True)
test['Alley'].fillna('No', inplace=True)

# label encode 1/0
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['Alley'] = le.fit_transform(train['Alley'])
test['Alley'] = le.transform(test['Alley'])

In [76]:
#LotShape non null
train.loc[(train['LotShape'] == 'IR2') | (train['LotShape'] == 'IR3') , 'LotShape'] = 'IR23'
test.loc[(test['LotShape'] == 'IR2') | (test['LotShape'] == 'IR3') , 'LotShape'] = 'IR23'

#encode
train = pd.concat([train, pd.get_dummies(train['LotShape'], drop_first=True, prefix='LotShape')], axis = 1)
train=train.drop(columns=['LotShape'])
test = pd.concat([test, pd.get_dummies(test['LotShape'], drop_first=True, prefix='LotShape')], axis = 1)
test=test.drop(columns=['LotShape'])

In [77]:
#LandContour. non null.
#encode
train = pd.concat([train, pd.get_dummies(train['LandContour'], drop_first=True, prefix='LandContour')], axis = 1)
train=train.drop(columns=['LandContour'])
test = pd.concat([test, pd.get_dummies(test['LandContour'], drop_first=True, prefix='LandContour')], axis = 1)
test=test.drop(columns=['LandContour'])

In [78]:
#LotConfig. combine inside and cfr
train.loc[(train['LotConfig'] == 'FR3') | (train['LotConfig'] == 'FR2') | (train['LotConfig'] == 'Corner'), 'LotConfig'] = 'Inside'
test.loc[(test['LotConfig'] == 'FR3') | (test['LotConfig'] == 'FR2') | (test['LotConfig'] == 'Corner'), 'LotConfig'] = 'Inside'

#encode
train = pd.concat([train, pd.get_dummies(train['LotConfig'], drop_first=True, prefix='LotConfig')], axis = 1)
train=train.drop(columns=['LotConfig'])
test = pd.concat([test, pd.get_dummies(test['LotConfig'], drop_first=True, prefix='LotConfig')], axis = 1)
test=test.drop(columns=['LotConfig'])

In [79]:
#LandSlope non null.
train.loc[(train['LandSlope'] == 'Sev'), 'LandSlope'] = 'Mod'
test.loc[(test['LandSlope'] == 'Sev'), 'LandSlope'] = 'Mod'

#label encode
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
train['LandSlope'] = le.fit_transform(train['LandSlope'])
test['LandSlope'] = le.transform(test['LandSlope'])

In [80]:
#Neighborhood
train['Neighborhood'].replace(to_replace='Veenker', value=np.nan, inplace=True) 
train['Neighborhood'].replace(to_replace='NPkVill', value=np.nan, inplace=True) 
train['Neighborhood'].replace(to_replace='Blueste', value=np.nan, inplace=True) 
train['Neighborhood'].replace(to_replace='BrDale', value=np.nan, inplace=True) 
train['Neighborhood'].replace(to_replace='Blmngtn', value=np.nan, inplace=True) 

test['Neighborhood'].replace(to_replace='Veenker', value=np.nan, inplace=True) 
test['Neighborhood'].replace(to_replace='NPkVill', value=np.nan, inplace=True) 
test['Neighborhood'].replace(to_replace='Blueste', value=np.nan, inplace=True) 
test['Neighborhood'].replace(to_replace='BrDale', value=np.nan, inplace=True) 
test['Neighborhood'].replace(to_replace='Blmngtn', value=np.nan, inplace=True) 

#encode
train = pd.concat([train, pd.get_dummies(train['Neighborhood'], prefix='Neighborhood')], axis = 1)
train=train.drop(columns=['Neighborhood'])
test = pd.concat([test, pd.get_dummies(test['Neighborhood'],  prefix='Neighborhood')], axis = 1)
test=test.drop(columns=['Neighborhood'])

In [81]:
#Condition1 and 2. 1)Drop cond2. 2)make new col- 1,2//1,1

train.loc[(train['Condition1'] == 'Norm') | (train['Condition2'] == 'Norm') , 'Cond_Norm'] = 1
train.loc[(train['Condition1'] == 'Feedr') | (train['Condition2'] == 'Feedr') , 'Cond_Feedr'] = 1
train.loc[(train['Condition1'] == 'Artery') | (train['Condition2'] == 'Artery') , 'Cond_Artery'] = 1
train.loc[(train['Condition1'] == 'RRAn') | (train['Condition2'] == 'RRAn') , 'Cond_RRAn'] = 1
train.loc[(train['Condition1'] == 'PosN') | (train['Condition2'] == 'PosN') , 'Cond_PosN'] = 1
train.loc[(train['Condition1'] == 'RRAe') | (train['Condition2'] == 'RRAe') , 'Cond_RRAe'] = 1

test.loc[(test['Condition1'] == 'Norm') | (test['Condition2'] == 'Norm') , 'Cond_Norm'] = 1
test.loc[(test['Condition1'] == 'Feedr') | (test['Condition2'] == 'Feedr') , 'Cond_Feedr'] = 1
test.loc[(test['Condition1'] == 'Artery') | (test['Condition2'] == 'Artery') , 'Cond_Artery'] = 1
test.loc[(test['Condition1'] == 'RRAn') | (test['Condition2'] == 'RRAn') , 'Cond_RRAn'] = 1
test.loc[(test['Condition1'] == 'PosN') | (test['Condition2'] == 'PosN') , 'Cond_PosN'] = 1
test.loc[(test['Condition1'] == 'RRAe') | (test['Condition2'] == 'RRAe') , 'Cond_RRAe'] = 1

train=train.drop(columns=['Condition1','Condition2'])
test=test.drop(columns=['Condition1','Condition2'])

train[['Cond_Norm','Cond_Feedr','Cond_Artery','Cond_RRAn','Cond_PosN','Cond_RRAe']] = train[['Cond_Norm','Cond_Feedr','Cond_Artery','Cond_RRAn','Cond_PosN','Cond_RRAe']].fillna(0).astype(int)
test[['Cond_Norm','Cond_Feedr','Cond_Artery','Cond_RRAn','Cond_PosN','Cond_RRAe']] = test[['Cond_Norm','Cond_Feedr','Cond_Artery','Cond_RRAn','Cond_PosN','Cond_RRAe']].fillna(0).astype(int)

In [82]:
#BldgType
#encode
train = pd.concat([train, pd.get_dummies(train['BldgType'], drop_first=True, prefix='BldgType')], axis = 1)
train=train.drop(columns=['BldgType'])
test = pd.concat([test, pd.get_dummies(test['BldgType'], drop_first=True, prefix='BldgType')], axis = 1)
test=test.drop(columns=['BldgType'])

In [83]:
#HouseStyle
train.loc[(train['HouseStyle'] == '2.5Fin') | (train['HouseStyle'] == '2.5Unf'), 'HouseStyle'] = '2.5Story'
test.loc[(test['HouseStyle'] == '2.5Fin') | (test['HouseStyle'] == '2.5Unf') , 'HouseStyle'] = '2.5Story'
train.loc[(train['HouseStyle'] == '1.5Fin') | (train['HouseStyle'] == '1.5Unf'), 'HouseStyle'] = '1.5Story'
test.loc[(test['HouseStyle'] == '1.5Fin') | (test['HouseStyle'] == '1.5Unf') , 'HouseStyle'] = '1.5Story'
#encode
train = pd.concat([train, pd.get_dummies(train['HouseStyle'], drop_first=True, prefix='HouseStyle')], axis = 1)
train=train.drop(columns=['HouseStyle'])
test = pd.concat([test, pd.get_dummies(test['HouseStyle'], drop_first=True, prefix='HouseStyle')], axis = 1)
test=test.drop(columns=['HouseStyle'])

In [84]:
#YearBuilt, YearRemodAdd bands
bins = pd.IntervalIndex.from_tuples([(1871, 1911), (1911, 1931), (1931, 1950), (1950,1970), (1970, 1990), (1990,2000), (2000,2010)])
train['YearBuilt']=pd.cut(train['YearBuilt'], bins=bins)
test['YearBuilt']=pd.cut(test['YearBuilt'], bins=bins)

from sklearn import preprocessing
le_yb = preprocessing.LabelEncoder()
train['YearBuilt'] = le_yb.fit_transform(train['YearBuilt'])
test['YearBuilt'] = le_yb.transform(test['YearBuilt'])

In [85]:
#RoofStyle
train.loc[(train['RoofStyle'] == 'Shed') | (train['RoofStyle'] == 'Mansard') | (train['RoofStyle'] == 'Gambrel') | (train['RoofStyle'] == 'Flat'), 'RoofStyle'] = 'Other'
test.loc[(test['RoofStyle'] == 'Shed') | (test['RoofStyle'] == 'Mansard') | (test['RoofStyle'] == 'Gambrel') | (test['RoofStyle'] == 'Flat'), 'RoofStyle'] = 'Other'

#encode
train = pd.concat([train, pd.get_dummies(train['RoofStyle'], drop_first=True, prefix='RoofStyle')], axis = 1)
train=train.drop(columns=['RoofStyle'])
test = pd.concat([test, pd.get_dummies(test['RoofStyle'], drop_first=True, prefix='RoofStyle')], axis = 1)
test=test.drop(columns=['RoofStyle'])

In [86]:
#RoofMatl,
train.loc[(train['RoofMatl'] != 'CompShg'), 'RoofMatl'] = 0
train.loc[(train['RoofMatl'] == 'CompShg'), 'RoofMatl'] = 1

test.loc[(test['RoofMatl'] != 'CompShg'), 'RoofMatl'] = 0
test.loc[(test['RoofMatl'] == 'CompShg'), 'RoofMatl'] = 1

train['RoofMatl']=train['RoofMatl'].astype(int)
test['RoofMatl']=test['RoofMatl'].astype(int)

In [87]:
#Exterior1st Exterior2nd missing in test

train['Exterior1st'].value_counts()

test['Exterior1st'] = test['Exterior1st'].fillna(test['Exterior1st'].value_counts().index[0])
test['Exterior2nd']= test['Exterior2nd'].fillna(test['Exterior2nd'].value_counts().index[0])
train.loc[(train['Exterior1st'] == 'VinylSd') | (train['Exterior2nd'] == 'VinylSd') , 'Ext_VinylSd'] = 1
train.loc[(train['Exterior1st'] == 'HdBoard') | (train['Exterior2nd'] == 'HdBoard') , 'Ext_HdBoard'] = 1
train.loc[(train['Exterior1st'] == 'MetalSd') | (train['Exterior2nd'] == 'MetalSd') , 'Ext_MetalSd'] = 1
train.loc[(train['Exterior1st'] == 'Wd Sdng') | (train['Exterior2nd'] == 'Wd Sdng') , 'Ext_Wd_Sdng'] = 1
train.loc[(train['Exterior1st'] == 'Wd Shng') | (train['Exterior2nd'] == 'Wd Shng') , 'Ext_Wd_Shng'] = 1
train.loc[(train['Exterior1st'] == 'Plywood') | (train['Exterior2nd'] == 'Plywood') , 'Ext_Plywood'] = 1
train.loc[(train['Exterior1st'] == 'CemntBd') | (train['Exterior2nd'] == 'CemntBd') , 'Ext_CemntBd'] = 1
train.loc[(train['Exterior1st'] == 'BrkFace') | (train['Exterior2nd'] == 'BrkFace') , 'Ext_BrkFace'] = 1
train.loc[(train['Exterior1st'] == 'WdShing') | (train['Exterior2nd'] == 'WdShing'), 'Ext_WdShing'] = 1
train.loc[(train['Exterior1st'] == 'Stucco') | (train['Exterior2nd'] == 'Stucco') , 'Ext_Stucco'] = 1
train.loc[(train['Exterior1st'] == 'AsbShng') | (train['Exterior2nd'] == 'AsbShng') , 'Ext_AsbShng'] = 1

test.loc[(test['Exterior1st'] == 'VinylSd') | (test['Exterior2nd'] == 'VinylSd') , 'Ext_VinylSd'] = 1
test.loc[(test['Exterior1st'] == 'HdBoard') | (test['Exterior2nd'] == 'HdBoard') , 'Ext_HdBoard'] = 1
test.loc[(test['Exterior1st'] == 'MetalSd') | (test['Exterior2nd'] == 'MetalSd') , 'Ext_MetalSd'] = 1
test.loc[(test['Exterior1st'] == 'Wd Sdng') | (test['Exterior2nd'] == 'Wd Sdng') , 'Ext_Wd_Sdng'] = 1
test.loc[(test['Exterior1st'] == 'Wd Shng') | (test['Exterior2nd'] == 'Wd Shng') , 'Ext_Wd_Shng'] = 1
test.loc[(test['Exterior1st'] == 'Plywood') | (test['Exterior2nd'] == 'Plywood') , 'Ext_Plywood'] = 1
test.loc[(test['Exterior1st'] == 'CemntBd') | (test['Exterior2nd'] == 'CemntBd') , 'Ext_CemntBd'] = 1
test.loc[(test['Exterior1st'] == 'BrkFace') | (test['Exterior2nd'] == 'BrkFace') , 'Ext_BrkFace'] = 1
test.loc[(test['Exterior1st'] == 'WdShing') | (test['Exterior2nd'] == 'WdShing'), 'Ext_WdShing'] = 1
test.loc[(test['Exterior1st'] == 'Stucco') | (test['Exterior2nd'] == 'Stucco') , 'Ext_Stucco'] = 1
test.loc[(test['Exterior1st'] == 'AsbShng') | (test['Exterior2nd'] == 'AsbShng') , 'Ext_AsbShng'] = 1


train[['Ext_VinylSd','Ext_HdBoard','Ext_MetalSd','Ext_Wd_Sdng','Ext_Wd_Shng','Ext_Plywood', 'Ext_CemntBd', 'Ext_BrkFace', 'Ext_WdShing', 'Ext_Stucco', 'Ext_AsbShng']]=train[['Ext_VinylSd','Ext_HdBoard','Ext_MetalSd','Ext_Wd_Sdng','Ext_Wd_Shng','Ext_Plywood', 'Ext_CemntBd', 'Ext_BrkFace', 'Ext_WdShing', 'Ext_Stucco', 'Ext_AsbShng']].fillna(0).astype(int)
test[['Ext_VinylSd','Ext_HdBoard','Ext_MetalSd','Ext_Wd_Sdng','Ext_Wd_Shng','Ext_Plywood', 'Ext_CemntBd', 'Ext_BrkFace', 'Ext_WdShing', 'Ext_Stucco', 'Ext_AsbShng']]=test[['Ext_VinylSd','Ext_HdBoard','Ext_MetalSd','Ext_Wd_Sdng','Ext_Wd_Shng','Ext_Plywood', 'Ext_CemntBd', 'Ext_BrkFace', 'Ext_WdShing', 'Ext_Stucco', 'Ext_AsbShng']].fillna(0).astype(int)

train[['Ext_VinylSd','Ext_HdBoard','Ext_MetalSd','Ext_Wd_Sdng','Ext_Wd_Shng','Ext_Plywood', 'Ext_CemntBd', 'Ext_BrkFace', 'Ext_WdShing', 'Ext_Stucco', 'Ext_AsbShng']].isna().sum()


train=train.drop(columns=['Exterior1st','Exterior2nd'])
test=test.drop(columns=['Exterior1st','Exterior2nd'])

In [88]:
#MasVnrType drop BrkCmn
train.loc[(train['MasVnrType'] == 'BrkCmn') | (train['MasVnrType'] == 'BrkCmn') , 'MasVnrType'] = np.nan
test.loc[(test['MasVnrType'] == 'BrkCmn') | (test['MasVnrType'] == 'BrkCmn') , 'MasVnrType'] = np.nan

#missing values
train['MasVnrType'] = train['MasVnrType'].fillna(train['MasVnrType'].value_counts().index[0])
test['MasVnrType'] = train['MasVnrType'].fillna(test['MasVnrType'].value_counts().index[0])

# #encode
train = pd.concat([train, pd.get_dummies(train['MasVnrType'], drop_first=True, prefix='MasVnrType')], axis = 1)
train=train.drop(columns=['MasVnrType'])
test = pd.concat([test, pd.get_dummies(test['MasVnrType'], drop_first=True, prefix='MasVnrType')], axis = 1)
test=test.drop(columns=['MasVnrType'])

In [89]:
#MasVnrArea
#missing values
train['MasVnrArea'] = train['MasVnrArea'].fillna(train['MasVnrArea'].value_counts().index[0])
test['MasVnrArea'] = test['MasVnrArea'].fillna(train['MasVnrArea'].value_counts().index[0])

#scaled
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train['MasVnrArea'] = sc.fit_transform(train['MasVnrArea'].values.reshape(-1, 1))
test['MasVnrArea'] = sc.transform(test['MasVnrArea'].values.reshape(-1, 1))

In [90]:
#ExterQual
#encode
train['ExterQual']=train['ExterQual'].map({
    'TA': 1,
    'Gd': 2,
    'Ex': 3,
    'Fa': 0})

test['ExterQual']=test['ExterQual'].map({
    'TA': 1,
    'Gd': 2,
    'Ex': 3,
    'Fa': 0})

In [91]:
#ExterCond
#remove Po,Ex
#encode
train['ExterCond']=train['ExterCond'].map({
    'TA': 2,
    'Gd': 3,
    'Ex': 4,
    'Fa': 1,
    'Po': 0})
test['ExterCond']=test['ExterCond'].map({
    'TA': 2,
    'Gd': 3,
    'Ex': 4,
    'Fa': 1,
    'Po': 0})

In [92]:
#Foundation

train.loc[(train['Foundation'] == 'Stone') | (train['Foundation'] == 'Wood') , 'Foundation'] = train['Foundation'].value_counts().index[0]
test.loc[(test['Foundation'] == 'Stone') | (test['Foundation'] == 'Wood') , 'Foundation'] = train['Foundation'].value_counts().index[0]
train['Foundation'].value_counts()
#encode
train = pd.concat([train, pd.get_dummies(train['Foundation'], drop_first=True, prefix='Foundation')], axis = 1)
train=train.drop(columns=['Foundation'])
test = pd.concat([test, pd.get_dummies(test['Foundation'], drop_first=True, prefix='Foundation')], axis = 1)
test=test.drop(columns=['Foundation'])

In [93]:
#BsmtQual 

#missing values == no basement
train['BsmtQual'] = train['BsmtQual'].fillna('No')
test['BsmtQual'] = test['BsmtQual'].fillna('No')

#encode
train['BsmtQual']=train['BsmtQual'].map({
    'TA': 3,
    'Gd': 4,
    'Ex': 5,
    'Fa': 2,
    'Po': 1,
    'No': 0,})

test['BsmtQual']=test['BsmtQual'].map({
    'TA': 3,
    'Gd': 4,
    'Ex': 5,
    'Fa': 2,
    'Po': 1,
    'No': 0,})

In [94]:
#BsmtCond
#missing values == no basement
train['BsmtCond'] = train['BsmtCond'].fillna('No')
test['BsmtCond'] = test['BsmtCond'].fillna('No')

#encode
train['BsmtCond']=train['BsmtCond'].map({
    'TA': 3,
    'Gd': 4,
    'Ex': 5,
    'Fa': 2,
    'Po': 1,
    'No': 0,})

test['BsmtCond']=test['BsmtCond'].map({
    'TA': 3,
    'Gd': 4,
    'Ex': 5,
    'Fa': 2,
    'Po': 1,
    'No': 0,})

In [95]:
#BsmtExposure
#missing values == no basement
train['BsmtExposure'] = train['BsmtExposure'].fillna('NoB')
test['BsmtExposure'] = test['BsmtExposure'].fillna('NoB')

train['BsmtExposure']=train['BsmtExposure'].map({
    'Gd': 4,
    'Av': 3,
    'Mn': 2,
    'No': 1,
    'NoB': 0,})
test['BsmtExposure']=test['BsmtExposure'].map({
    'Gd': 4,
    'Av': 3,
    'Mn': 2,
    'No': 1,
    'NoB': 0,})

In [96]:
#Bsmts, I combined category and feet of that category

#missing values == no basement
train['BsmtFinType1'] = train['BsmtFinType1'].fillna('No')
test['BsmtFinType1'] = test['BsmtFinType1'].fillna('No')
train['BsmtFinType2'] = train['BsmtFinType2'].fillna('No')
test['BsmtFinType2'] = test['BsmtFinType2'].fillna('No')

test['BsmtFinSF1']=test['BsmtFinSF1'].fillna(train['BsmtFinSF1'].mean())
test['BsmtFinSF2']=test['BsmtFinSF2'].fillna(train['BsmtFinSF2'].mean())

train.loc[(train['BsmtFinType1'] == 'GLQ'), 'Bsmt_GLQ'] = train['BsmtFinSF1']
train.loc[(train['BsmtFinType1'] == 'ALQ'), 'Bsmt_ALQ'] = train['BsmtFinSF1']
train.loc[(train['BsmtFinType1'] == 'BLQ'), 'Bsmt_BLQ'] = train['BsmtFinSF1']
train.loc[(train['BsmtFinType1'] == 'Rec'), 'Bsmt_Rec'] = train['BsmtFinSF1']
train.loc[(train['BsmtFinType1'] == 'LwQ'), 'Bsmt_LwQ'] = train['BsmtFinSF1']

test.loc[(test['BsmtFinType1'] == 'GLQ'), 'Bsmt_GLQ'] = test['BsmtFinSF1']
test.loc[(test['BsmtFinType1'] == 'ALQ'), 'Bsmt_ALQ'] = test['BsmtFinSF1']
test.loc[(test['BsmtFinType1'] == 'BLQ'), 'Bsmt_BLQ'] = test['BsmtFinSF1']
test.loc[(test['BsmtFinType1'] == 'Rec'), 'Bsmt_Rec'] = test['BsmtFinSF1']
test.loc[(test['BsmtFinType1'] == 'LwQ'), 'Bsmt_LwQ'] = test['BsmtFinSF1']

train.loc[(train['BsmtFinType2'] == 'GLQ'), 'Bsmt_GLQ'] = train['BsmtFinSF2']
train.loc[(train['BsmtFinType2'] == 'ALQ'), 'Bsmt_ALQ'] = train['BsmtFinSF2']
train.loc[(train['BsmtFinType2'] == 'BLQ'), 'Bsmt_BLQ'] = train['BsmtFinSF2']
train.loc[(train['BsmtFinType2'] == 'Rec'), 'Bsmt_Rec'] = train['BsmtFinSF2']
train.loc[(train['BsmtFinType2'] == 'LwQ'), 'Bsmt_LwQ'] = train['BsmtFinSF2']

test.loc[(test['BsmtFinType2'] == 'GLQ'), 'Bsmt_GLQ'] = test['BsmtFinSF2']
test.loc[(test['BsmtFinType2'] == 'ALQ'), 'Bsmt_ALQ'] = test['BsmtFinSF2']
test.loc[(test['BsmtFinType2'] == 'BLQ'), 'Bsmt_BLQ'] = test['BsmtFinSF2']
test.loc[(test['BsmtFinType2'] == 'Rec'), 'Bsmt_Rec'] = test['BsmtFinSF2']
test.loc[(test['BsmtFinType2'] == 'LwQ'), 'Bsmt_LwQ'] = test['BsmtFinSF2']

train[['Bsmt_GLQ','Bsmt_ALQ','Bsmt_BLQ','Bsmt_Rec','Bsmt_LwQ']]=train[['Bsmt_GLQ','Bsmt_ALQ','Bsmt_BLQ','Bsmt_Rec','Bsmt_LwQ']].fillna(0).astype(int)
test[['Bsmt_GLQ','Bsmt_ALQ','Bsmt_BLQ','Bsmt_Rec','Bsmt_LwQ']]=test[['Bsmt_GLQ','Bsmt_ALQ','Bsmt_BLQ','Bsmt_Rec','Bsmt_LwQ']].fillna(0).astype(int)

train=train.drop(columns=['BsmtFinType1','BsmtFinType2', 'BsmtFinSF1', 'BsmtFinSF2'])
test=test.drop(columns=['BsmtFinType1','BsmtFinType2', 'BsmtFinSF1', 'BsmtFinSF2'])

test['BsmtUnfSF']=test['BsmtUnfSF'].fillna(0).astype(int)
test['TotalBsmtSF']=test['TotalBsmtSF'].fillna(0).astype(int)

In [97]:
#Heating
train.loc[(train['Heating'] != 'GasA') , 'Heating'] = 0
test.loc[(test['Heating'] != 'GasA') , 'Heating'] = 0
train.loc[(train['Heating'] == 'GasA') , 'Heating'] = 1
test.loc[(test['Heating'] == 'GasA') , 'Heating'] = 1

train['Heating']=train['Heating'].astype(int)
test['Heating']=test['Heating'].astype(int)

In [98]:
#HeatingQC
train['HeatingQC']=train['HeatingQC'].map({
    'Ex': 4,
    'Gd': 3,
    'TA': 2,
    'Fa': 1,
    'Po': 0,})

test['HeatingQC']=test['HeatingQC'].map({
    'Ex': 4,
    'Gd': 3,
    'TA': 2,
    'Fa': 1,
    'Po': 0,})

In [99]:
#CentralAir
#label encode 1/0
train['CentralAir'].replace(to_replace='Y', value=1, inplace=True) 
train['CentralAir'].replace(to_replace='N', value=0, inplace=True) 
test['CentralAir'].replace(to_replace='Y', value=1, inplace=True) 
test['CentralAir'].replace(to_replace='N', value=0, inplace=True)

In [100]:
#Electrical 1 null in train
# FuseF FuseP Mix
train.loc[(train['Electrical'] == 'FuseF') | (train['Electrical'] == 'FuseP') | (train['Electrical'] == 'Mix'), 'Electrical'] = 0
train['Electrical'].replace(to_replace='FuseA', value=1, inplace=True) 
train['Electrical'].replace(to_replace='SBrkr', value=2, inplace=True) 
train['Electrical']=train['Electrical'].fillna(0).astype(int)

test.loc[(test['Electrical'] == 'FuseF') | (test['Electrical'] == 'FuseP') | (test['Electrical'] == 'Mix'), 'Electrical'] = 0
test['Electrical'].replace(to_replace='FuseA', value=1, inplace=True) 
test['Electrical'].replace(to_replace='SBrkr', value=2, inplace=True) 
test['Electrical']=test['Electrical'].fillna(0).astype(int)

In [101]:
#LowQualFinSF drop?
train.loc[(train['LowQualFinSF'] != 0, 'LowQualFinSF')] = 1
test.loc[(test['LowQualFinSF'] != 0, 'LowQualFinSF')] = 1

In [102]:
#BsmtFullBath
#null in test.
test['BsmtFullBath']=test['BsmtFullBath'].fillna(0).astype(int)

train['BsmtFullBath'].replace(to_replace=3, value=0, inplace=True)
test['BsmtFullBath'].replace(to_replace=3, value=0, inplace=True)

In [103]:
#BsmtHalfBath
#null in test.
test['BsmtHalfBath']=test['BsmtHalfBath'].fillna(0).astype(int)

train['BsmtHalfBath'].replace(to_replace=2, value=0, inplace=True)
test['BsmtHalfBath'].replace(to_replace=2, value=0, inplace=True)

In [104]:
#FullBath combined 0 and 1
train['FullBath'].replace(to_replace=0, value=1, inplace=True)
test['FullBath'].replace(to_replace=0, value=1, inplace=True)

In [105]:
#HalfBath combined 2 and 1
train['HalfBath'].replace(to_replace=2, value=1, inplace=True)
test['HalfBath'].replace(to_replace=2, value=1, inplace=True)

In [106]:
#Bedroom removed 6 0 5 8. why is 1 higher than 2?
train['BedroomAbvGr'].replace(to_replace=6, value=3, inplace=True)
train['BedroomAbvGr'].replace(to_replace=8, value=3, inplace=True)
train['BedroomAbvGr'].replace(to_replace=5, value=3, inplace=True)
train['BedroomAbvGr'].replace(to_replace=0, value=3, inplace=True)

test['BedroomAbvGr'].replace(to_replace=6, value=3, inplace=True)
test['BedroomAbvGr'].replace(to_replace=8, value=3, inplace=True)
test['BedroomAbvGr'].replace(to_replace=5, value=3, inplace=True)
test['BedroomAbvGr'].replace(to_replace=0, value=3, inplace=True)

In [107]:
#KitchenAbvGr combined 0,1 and 2,3
train['KitchenAbvGr'].replace(to_replace=0, value=1, inplace=True)
train['KitchenAbvGr'].replace(to_replace=3, value=2, inplace=True)
test['KitchenAbvGr'].replace(to_replace=0, value=1, inplace=True)
test['KitchenAbvGr'].replace(to_replace=3, value=2, inplace=True)

In [108]:
#KitchenQual null in test
test['KitchenQual']=test['KitchenQual'].fillna(train['KitchenQual'].value_counts().index[0])
train['KitchenQual']=train['KitchenQual'].map({
    'Ex': 4,
    'Gd': 3,
    'TA': 2,
    'Fa': 1,
    'Po': 0,})
test['KitchenQual']=test['KitchenQual'].map({
    'Ex': 4,
    'Gd': 3,
    'TA': 2,
    'Fa': 1,
    'Po': 0,})

In [109]:
#TotRmsAbvGrd. combine 11+ and 3-

train.loc[(train['TotRmsAbvGrd'] >11), 'TotRmsAbvGrd'] = 11
train.loc[(train['TotRmsAbvGrd'] <3), 'TotRmsAbvGrd'] = 3
test.loc[(test['TotRmsAbvGrd'] >11), 'TotRmsAbvGrd'] = 11
test.loc[(test['TotRmsAbvGrd'] <3), 'TotRmsAbvGrd'] = 3

In [110]:
#Functional
#null in test
test['Functional']=test['Functional'].fillna('Typ')

train['Functional']=train['Functional'].map({
    'Typ': 4,
    'Min1': 3,
    'Min2': 2,
    'Mod': 1,
    'Maj1': 0,
    'Maj2': 0,
    'Sev': 0,
})


test['Functional']=test['Functional'].map({
    'Typ': 4,
    'Min1': 3,
    'Min2': 2,
    'Mod': 1,
    'Maj1': 0,
    'Maj2': 0,
    'Sev': 0,
})

In [111]:
#FireplaceQu
train['FireplaceQu']=train['FireplaceQu'].fillna('No')
test['FireplaceQu']=test['FireplaceQu'].fillna('No')
train['FireplaceQu']=train['FireplaceQu'].map({
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 0,
    'No': 1,
})
test['FireplaceQu']=test['FireplaceQu'].map({
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 0,
    'No': 1,
})

In [112]:
#GarageType 

train['GarageType']=train['GarageType'].fillna('No')
test['GarageType']=test['GarageType'].fillna('No')

train.loc[(train['GarageType'] == 'CarPort') | (train['GarageType'] == '2Types') , 'GarageType'] = 'Other'
test.loc[(test['GarageType'] == 'CarPort') | (test['GarageType'] == '2Types') , 'GarageType'] = 'Other'

#encode
train = pd.concat([train, pd.get_dummies(train['GarageType'], prefix='GarageType')], axis = 1)
train=train.drop(columns=['GarageType', 'GarageType_Other'])
test = pd.concat([test, pd.get_dummies(test['GarageType'],prefix='GarageType')], axis = 1)
test=test.drop(columns=['GarageType', 'GarageType_Other'])

In [113]:
#GarageYrBlt
#null values
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=5)
train['GarageYrBlt'] = imputer.fit_transform(train['GarageYrBlt'].values.reshape(-1, 1))
test['GarageYrBlt'] = imputer.transform(test['GarageYrBlt'].values.reshape(-1, 1))

bins = pd.IntervalIndex.from_tuples([(1700, 1960), (1960, 1985), (1985,2000), (2000, 2005), (2005,3000)])
train['GarageYrBlt']=pd.cut(train['GarageYrBlt'], bins=bins)
test['GarageYrBlt']=pd.cut(test['GarageYrBlt'], bins=bins)
train['GarageYrBlt'] = train['GarageYrBlt'].cat.codes
test['GarageYrBlt'] = test['GarageYrBlt'].cat.codes

In [114]:
#GarageFinish
train['GarageFinish']=train['GarageFinish'].fillna('No')
test['GarageFinish']=test['GarageFinish'].fillna('No')

train['GarageFinish']=train['GarageFinish'].map({
    'Fin': 3,
    'RFn': 2,
    'Unf': 1,
    'No': 0,
})
test['GarageFinish']=test['GarageFinish'].map({
    'Fin': 3,
    'RFn': 2,
    'Unf': 1,
    'No': 0,
})

In [115]:
#GarageCars grouped 3,3+
#missing
test['GarageCars']=test['GarageCars'].fillna(train['GarageCars'].value_counts().index[0])

train.loc[(train['GarageCars'] >3 , 'GarageCars')] = 3
test.loc[(test['GarageCars'] >3 , 'GarageCars')] = 3

In [116]:
#GarageArea
#missing
train['GarageArea']=train['GarageArea'].fillna(train['GarageCars'].mean())
test['GarageArea']=test['GarageArea'].fillna(train['GarageCars'].mean())

In [117]:
#GarageQual
#missing=no garage
train['GarageQual']=train['GarageQual'].fillna('No')
test['GarageQual']=test['GarageQual'].fillna('No')
train['GarageQual']=train['GarageQual'].map({
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 0,
    'No': 1,
})
test['GarageQual']=test['GarageQual'].map({
    'Ex': 5,
    'Gd': 4,
    'TA': 3,
    'Fa': 2,
    'Po': 0,
    'No': 1,
})

In [118]:
#GarageCond
#missing=no garage
train['GarageCond']=train['GarageCond'].fillna('No')
test['GarageCond']=test['GarageCond'].fillna('No')
train['GarageCond']=train['GarageCond'].map({
    'Ex': 2,
    'Gd': 2,
    'TA': 2,
    'Fa': 1,
    'Po': 1,
    'No': 0,
})
test['GarageCond']=test['GarageCond'].map({
    'Ex': 2,
    'Gd': 2,
    'TA': 2,
    'Fa': 1,
    'Po': 1,
    'No': 0,
})

In [119]:
#PavedDrive
train['PavedDrive']=train['PavedDrive'].map({
    'Y': 2,
    'P': 1,
    'N': 0,
})
test['PavedDrive']=test['PavedDrive'].map({
    'Y': 2,
    'P': 1,
    'N': 0,
})

In [120]:
#WoodDeckSF bands
bins = pd.IntervalIndex.from_tuples([(-1, 0), (0, 100), (100, 150), (150,20000)])

test['WoodDeckSF']=test['WoodDeckSF'].fillna(train['WoodDeckSF'].mean())

train['WoodDeckSF']=pd.cut(train['WoodDeckSF'], bins=bins)
test['WoodDeckSF']=pd.cut(test['WoodDeckSF'], bins=bins)

from sklearn import preprocessing
le_wd = preprocessing.LabelEncoder()
train['WoodDeckSF'] = le_wd.fit_transform(train['WoodDeckSF'])
test['WoodDeckSF'] = le_wd.transform(test['WoodDeckSF'])

In [121]:
#combine porch
train["Porch"] = train["OpenPorchSF"] + train["EnclosedPorch"] + train["3SsnPorch"] + train["ScreenPorch"]
test["Porch"] = test["OpenPorchSF"] + test["EnclosedPorch"] + test["3SsnPorch"] + test["ScreenPorch"]

train=train.drop(columns=['OpenPorchSF','EnclosedPorch', '3SsnPorch', 'ScreenPorch'])
test=test.drop(columns=['OpenPorchSF','EnclosedPorch', '3SsnPorch', 'ScreenPorch'])

In [122]:
#PoolArea and PoolQC dropped
train=train.drop(columns=['PoolArea','PoolQC'])
test=test.drop(columns=['PoolArea','PoolQC'])

In [123]:
#Fence drop
train['Fence'].fillna(0, inplace=True)
test['Fence'].fillna(0, inplace=True)

train.loc[(train['Fence'] != 0) , 'Fence'] = 1
test.loc[(test['Fence'] != 0) , 'Fence'] = 1

train['Fence']=train['Fence'].astype(int)
test['Fence']=test['Fence'].astype(int)

In [124]:
#MiscFeature drop
train=train.drop(columns=['MiscFeature'])
test=test.drop(columns=['MiscFeature'])

In [125]:
#SaleType drop. overfit
train=train.drop(columns=['SaleType'])
test=test.drop(columns=['SaleType'])

In [126]:
#SaleCondition Alloca AdjLand
train.loc[(train['SaleCondition'] != 'Partial'), 'SaleCondition'] = 0
train.loc[(train['SaleCondition'] == 'Partial'), 'SaleCondition'] = 1

test.loc[(test['SaleCondition'] != 'Partial'), 'SaleCondition'] = 0
test.loc[(test['SaleCondition'] == 'Partial'), 'SaleCondition'] = 1

train['SaleCondition']=train['SaleCondition'].astype(int)
test['SaleCondition']=test['SaleCondition'].astype(int)

In [127]:
#scale
from sklearn.preprocessing import StandardScaler
sc_sp = StandardScaler()
train['SalePrice'] = sc_sp.fit_transform(train['SalePrice'].values.reshape(-1, 1))

In [128]:
y_train=train['SalePrice'].iloc[:].values
x_train=train.drop(columns=['SalePrice'])
x_test=test

In [129]:
from numpy import mean
from numpy import std
from catboost import CatBoostRegressor
best_grid = CatBoostRegressor()
best_grid.fit(x_train, y_train)
round(best_grid.score(x_train, y_train), 2)

Learning rate set to 0.041745
0:	learn: 0.9739604	total: 4.63ms	remaining: 4.63s
1:	learn: 0.9495308	total: 8.02ms	remaining: 4s
2:	learn: 0.9248166	total: 12ms	remaining: 3.99s
3:	learn: 0.9014095	total: 15.6ms	remaining: 3.88s
4:	learn: 0.8787943	total: 18.9ms	remaining: 3.76s
5:	learn: 0.8580063	total: 22.3ms	remaining: 3.7s
6:	learn: 0.8353059	total: 24.8ms	remaining: 3.52s
7:	learn: 0.8165302	total: 27.3ms	remaining: 3.39s
8:	learn: 0.7979272	total: 29.8ms	remaining: 3.28s
9:	learn: 0.7799332	total: 32.3ms	remaining: 3.19s
10:	learn: 0.7603250	total: 34.7ms	remaining: 3.12s
11:	learn: 0.7423478	total: 38.1ms	remaining: 3.13s
12:	learn: 0.7253273	total: 41.4ms	remaining: 3.14s
13:	learn: 0.7111473	total: 43.9ms	remaining: 3.09s
14:	learn: 0.6953650	total: 46.9ms	remaining: 3.08s
15:	learn: 0.6816467	total: 49.5ms	remaining: 3.04s
16:	learn: 0.6677453	total: 53ms	remaining: 3.06s
17:	learn: 0.6546818	total: 55.7ms	remaining: 3.04s
18:	learn: 0.6414426	total: 58.4ms	remaining: 3.02s


192:	learn: 0.2186274	total: 579ms	remaining: 2.42s
193:	learn: 0.2183813	total: 582ms	remaining: 2.42s
194:	learn: 0.2179254	total: 585ms	remaining: 2.42s
195:	learn: 0.2177229	total: 587ms	remaining: 2.41s
196:	learn: 0.2172440	total: 591ms	remaining: 2.41s
197:	learn: 0.2168213	total: 595ms	remaining: 2.41s
198:	learn: 0.2162513	total: 598ms	remaining: 2.41s
199:	learn: 0.2155949	total: 601ms	remaining: 2.4s
200:	learn: 0.2152699	total: 604ms	remaining: 2.4s
201:	learn: 0.2151737	total: 607ms	remaining: 2.4s
202:	learn: 0.2145945	total: 610ms	remaining: 2.4s
203:	learn: 0.2142628	total: 613ms	remaining: 2.39s
204:	learn: 0.2137653	total: 617ms	remaining: 2.39s
205:	learn: 0.2133079	total: 620ms	remaining: 2.39s
206:	learn: 0.2129943	total: 623ms	remaining: 2.39s
207:	learn: 0.2125942	total: 626ms	remaining: 2.38s
208:	learn: 0.2122030	total: 629ms	remaining: 2.38s
209:	learn: 0.2119091	total: 632ms	remaining: 2.38s
210:	learn: 0.2117321	total: 634ms	remaining: 2.37s
211:	learn: 0.21

356:	learn: 0.1682856	total: 1.16s	remaining: 2.09s
357:	learn: 0.1680530	total: 1.16s	remaining: 2.08s
358:	learn: 0.1678717	total: 1.17s	remaining: 2.09s
359:	learn: 0.1677419	total: 1.17s	remaining: 2.09s
360:	learn: 0.1675441	total: 1.18s	remaining: 2.09s
361:	learn: 0.1672985	total: 1.18s	remaining: 2.09s
362:	learn: 0.1670608	total: 1.19s	remaining: 2.09s
363:	learn: 0.1668307	total: 1.2s	remaining: 2.09s
364:	learn: 0.1665355	total: 1.2s	remaining: 2.09s
365:	learn: 0.1662763	total: 1.21s	remaining: 2.1s
366:	learn: 0.1658628	total: 1.22s	remaining: 2.1s
367:	learn: 0.1656047	total: 1.22s	remaining: 2.1s
368:	learn: 0.1652869	total: 1.22s	remaining: 2.09s
369:	learn: 0.1651031	total: 1.23s	remaining: 2.09s
370:	learn: 0.1648615	total: 1.23s	remaining: 2.09s
371:	learn: 0.1644961	total: 1.24s	remaining: 2.08s
372:	learn: 0.1643339	total: 1.24s	remaining: 2.08s
373:	learn: 0.1640862	total: 1.24s	remaining: 2.08s
374:	learn: 0.1638264	total: 1.25s	remaining: 2.08s
375:	learn: 0.163

529:	learn: 0.1351665	total: 1.74s	remaining: 1.55s
530:	learn: 0.1349328	total: 1.75s	remaining: 1.54s
531:	learn: 0.1348470	total: 1.75s	remaining: 1.54s
532:	learn: 0.1346556	total: 1.75s	remaining: 1.54s
533:	learn: 0.1345190	total: 1.76s	remaining: 1.53s
534:	learn: 0.1343378	total: 1.76s	remaining: 1.53s
535:	learn: 0.1341611	total: 1.76s	remaining: 1.53s
536:	learn: 0.1340273	total: 1.77s	remaining: 1.52s
537:	learn: 0.1339152	total: 1.77s	remaining: 1.52s
538:	learn: 0.1337998	total: 1.77s	remaining: 1.52s
539:	learn: 0.1335332	total: 1.78s	remaining: 1.51s
540:	learn: 0.1333765	total: 1.78s	remaining: 1.51s
541:	learn: 0.1331004	total: 1.78s	remaining: 1.5s
542:	learn: 0.1330345	total: 1.78s	remaining: 1.5s
543:	learn: 0.1329066	total: 1.79s	remaining: 1.5s
544:	learn: 0.1328831	total: 1.79s	remaining: 1.49s
545:	learn: 0.1327602	total: 1.79s	remaining: 1.49s
546:	learn: 0.1325738	total: 1.79s	remaining: 1.49s
547:	learn: 0.1324414	total: 1.8s	remaining: 1.48s
548:	learn: 0.13

726:	learn: 0.1085577	total: 2.32s	remaining: 873ms
727:	learn: 0.1084740	total: 2.33s	remaining: 870ms
728:	learn: 0.1083235	total: 2.33s	remaining: 867ms
729:	learn: 0.1081282	total: 2.33s	remaining: 864ms
730:	learn: 0.1079554	total: 2.34s	remaining: 861ms
731:	learn: 0.1078482	total: 2.34s	remaining: 858ms
732:	learn: 0.1077432	total: 2.35s	remaining: 855ms
733:	learn: 0.1075749	total: 2.35s	remaining: 852ms
734:	learn: 0.1074319	total: 2.35s	remaining: 849ms
735:	learn: 0.1073375	total: 2.36s	remaining: 845ms
736:	learn: 0.1071953	total: 2.36s	remaining: 842ms
737:	learn: 0.1070939	total: 2.36s	remaining: 839ms
738:	learn: 0.1070211	total: 2.37s	remaining: 836ms
739:	learn: 0.1068998	total: 2.37s	remaining: 833ms
740:	learn: 0.1067949	total: 2.37s	remaining: 829ms
741:	learn: 0.1067844	total: 2.37s	remaining: 826ms
742:	learn: 0.1067326	total: 2.38s	remaining: 822ms
743:	learn: 0.1065962	total: 2.38s	remaining: 819ms
744:	learn: 0.1064918	total: 2.38s	remaining: 816ms
745:	learn: 

923:	learn: 0.0888324	total: 2.91s	remaining: 239ms
924:	learn: 0.0887106	total: 2.91s	remaining: 236ms
925:	learn: 0.0886044	total: 2.91s	remaining: 233ms
926:	learn: 0.0884585	total: 2.92s	remaining: 230ms
927:	learn: 0.0883856	total: 2.92s	remaining: 227ms
928:	learn: 0.0882453	total: 2.92s	remaining: 224ms
929:	learn: 0.0881522	total: 2.93s	remaining: 220ms
930:	learn: 0.0880636	total: 2.93s	remaining: 217ms
931:	learn: 0.0879495	total: 2.94s	remaining: 214ms
932:	learn: 0.0879074	total: 2.94s	remaining: 211ms
933:	learn: 0.0878015	total: 2.94s	remaining: 208ms
934:	learn: 0.0876754	total: 2.94s	remaining: 205ms
935:	learn: 0.0876060	total: 2.95s	remaining: 202ms
936:	learn: 0.0875392	total: 2.95s	remaining: 198ms
937:	learn: 0.0874620	total: 2.95s	remaining: 195ms
938:	learn: 0.0873618	total: 2.96s	remaining: 192ms
939:	learn: 0.0873199	total: 2.96s	remaining: 189ms
940:	learn: 0.0872151	total: 2.96s	remaining: 186ms
941:	learn: 0.0871179	total: 2.97s	remaining: 183ms
942:	learn: 

0.99

In [134]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(best_grid, x_train, y_train, scoring='r2', cv=5)
scores

Learning rate set to 0.040118
0:	learn: 0.9893108	total: 4.67ms	remaining: 4.67s
1:	learn: 0.9641475	total: 7.96ms	remaining: 3.97s
2:	learn: 0.9406486	total: 12ms	remaining: 4s
3:	learn: 0.9175811	total: 16.8ms	remaining: 4.19s
4:	learn: 0.8961471	total: 21.1ms	remaining: 4.19s
5:	learn: 0.8753186	total: 24.9ms	remaining: 4.12s
6:	learn: 0.8547357	total: 28.8ms	remaining: 4.08s
7:	learn: 0.8369999	total: 32.6ms	remaining: 4.04s
8:	learn: 0.8174423	total: 36.1ms	remaining: 3.98s
9:	learn: 0.7998159	total: 40.3ms	remaining: 3.99s
10:	learn: 0.7806356	total: 43.8ms	remaining: 3.94s
11:	learn: 0.7637854	total: 47ms	remaining: 3.87s
12:	learn: 0.7461086	total: 50.9ms	remaining: 3.87s
13:	learn: 0.7293524	total: 54.3ms	remaining: 3.82s
14:	learn: 0.7129939	total: 57.9ms	remaining: 3.8s
15:	learn: 0.6989638	total: 60.8ms	remaining: 3.74s
16:	learn: 0.6860084	total: 64.3ms	remaining: 3.72s
17:	learn: 0.6721127	total: 67.7ms	remaining: 3.69s
18:	learn: 0.6590945	total: 71ms	remaining: 3.67s
19

187:	learn: 0.2215895	total: 577ms	remaining: 2.49s
188:	learn: 0.2213330	total: 578ms	remaining: 2.48s
189:	learn: 0.2208834	total: 582ms	remaining: 2.48s
190:	learn: 0.2202178	total: 586ms	remaining: 2.48s
191:	learn: 0.2195332	total: 591ms	remaining: 2.48s
192:	learn: 0.2190811	total: 595ms	remaining: 2.49s
193:	learn: 0.2188357	total: 597ms	remaining: 2.48s
194:	learn: 0.2180550	total: 601ms	remaining: 2.48s
195:	learn: 0.2175267	total: 605ms	remaining: 2.48s
196:	learn: 0.2172389	total: 608ms	remaining: 2.48s
197:	learn: 0.2169506	total: 612ms	remaining: 2.48s
198:	learn: 0.2166920	total: 615ms	remaining: 2.47s
199:	learn: 0.2164013	total: 616ms	remaining: 2.46s
200:	learn: 0.2160595	total: 620ms	remaining: 2.46s
201:	learn: 0.2157909	total: 623ms	remaining: 2.46s
202:	learn: 0.2155379	total: 624ms	remaining: 2.45s
203:	learn: 0.2149919	total: 627ms	remaining: 2.45s
204:	learn: 0.2145469	total: 631ms	remaining: 2.45s
205:	learn: 0.2141316	total: 635ms	remaining: 2.45s
206:	learn: 

367:	learn: 0.1603573	total: 1.16s	remaining: 1.99s
368:	learn: 0.1602964	total: 1.16s	remaining: 1.98s
369:	learn: 0.1598922	total: 1.16s	remaining: 1.98s
370:	learn: 0.1596336	total: 1.17s	remaining: 1.98s
371:	learn: 0.1594050	total: 1.17s	remaining: 1.97s
372:	learn: 0.1591387	total: 1.17s	remaining: 1.97s
373:	learn: 0.1588442	total: 1.18s	remaining: 1.97s
374:	learn: 0.1585166	total: 1.18s	remaining: 1.96s
375:	learn: 0.1581810	total: 1.18s	remaining: 1.96s
376:	learn: 0.1577754	total: 1.18s	remaining: 1.96s
377:	learn: 0.1577287	total: 1.19s	remaining: 1.95s
378:	learn: 0.1575925	total: 1.19s	remaining: 1.95s
379:	learn: 0.1574529	total: 1.19s	remaining: 1.95s
380:	learn: 0.1572207	total: 1.2s	remaining: 1.94s
381:	learn: 0.1570890	total: 1.2s	remaining: 1.94s
382:	learn: 0.1569147	total: 1.2s	remaining: 1.94s
383:	learn: 0.1566165	total: 1.2s	remaining: 1.93s
384:	learn: 0.1565789	total: 1.21s	remaining: 1.93s
385:	learn: 0.1562906	total: 1.21s	remaining: 1.92s
386:	learn: 0.15

546:	learn: 0.1246525	total: 1.74s	remaining: 1.44s
547:	learn: 0.1245143	total: 1.74s	remaining: 1.44s
548:	learn: 0.1242807	total: 1.74s	remaining: 1.43s
549:	learn: 0.1240991	total: 1.75s	remaining: 1.43s
550:	learn: 0.1238917	total: 1.75s	remaining: 1.43s
551:	learn: 0.1237930	total: 1.76s	remaining: 1.43s
552:	learn: 0.1236289	total: 1.76s	remaining: 1.42s
553:	learn: 0.1234625	total: 1.76s	remaining: 1.42s
554:	learn: 0.1232967	total: 1.76s	remaining: 1.42s
555:	learn: 0.1231799	total: 1.77s	remaining: 1.41s
556:	learn: 0.1229676	total: 1.77s	remaining: 1.41s
557:	learn: 0.1228812	total: 1.78s	remaining: 1.41s
558:	learn: 0.1227768	total: 1.78s	remaining: 1.4s
559:	learn: 0.1227625	total: 1.78s	remaining: 1.4s
560:	learn: 0.1225266	total: 1.78s	remaining: 1.4s
561:	learn: 0.1222823	total: 1.79s	remaining: 1.39s
562:	learn: 0.1221255	total: 1.79s	remaining: 1.39s
563:	learn: 0.1219181	total: 1.79s	remaining: 1.39s
564:	learn: 0.1217669	total: 1.8s	remaining: 1.38s
565:	learn: 0.12

735:	learn: 0.0987897	total: 2.32s	remaining: 832ms
736:	learn: 0.0985848	total: 2.32s	remaining: 829ms
737:	learn: 0.0984607	total: 2.33s	remaining: 826ms
738:	learn: 0.0983718	total: 2.33s	remaining: 822ms
739:	learn: 0.0981898	total: 2.33s	remaining: 820ms
740:	learn: 0.0980957	total: 2.34s	remaining: 817ms
741:	learn: 0.0979988	total: 2.34s	remaining: 814ms
742:	learn: 0.0979148	total: 2.34s	remaining: 810ms
743:	learn: 0.0977691	total: 2.35s	remaining: 808ms
744:	learn: 0.0976847	total: 2.35s	remaining: 804ms
745:	learn: 0.0976028	total: 2.35s	remaining: 802ms
746:	learn: 0.0974723	total: 2.36s	remaining: 798ms
747:	learn: 0.0973010	total: 2.36s	remaining: 795ms
748:	learn: 0.0972245	total: 2.36s	remaining: 792ms
749:	learn: 0.0970702	total: 2.37s	remaining: 789ms
750:	learn: 0.0968816	total: 2.37s	remaining: 785ms
751:	learn: 0.0966747	total: 2.37s	remaining: 782ms
752:	learn: 0.0966454	total: 2.37s	remaining: 779ms
753:	learn: 0.0964786	total: 2.38s	remaining: 776ms
754:	learn: 

...

84:	learn: 0.3012522	total: 287ms	remaining: 3.09s
85:	learn: 0.2994453	total: 295ms	remaining: 3.14s
86:	learn: 0.2979214	total: 299ms	remaining: 3.14s
87:	learn: 0.2965252	total: 305ms	remaining: 3.16s
88:	learn: 0.2950800	total: 309ms	remaining: 3.16s
89:	learn: 0.2937477	total: 313ms	remaining: 3.16s
90:	learn: 0.2925927	total: 316ms	remaining: 3.16s
91:	learn: 0.2910774	total: 319ms	remaining: 3.15s
92:	learn: 0.2898929	total: 323ms	remaining: 3.15s
93:	learn: 0.2885039	total: 327ms	remaining: 3.15s
94:	learn: 0.2870494	total: 330ms	remaining: 3.15s
95:	learn: 0.2858725	total: 334ms	remaining: 3.15s
96:	learn: 0.2852430	total: 338ms	remaining: 3.15s
97:	learn: 0.2838052	total: 341ms	remaining: 3.14s
98:	learn: 0.2824571	total: 344ms	remaining: 3.13s
99:	learn: 0.2812564	total: 348ms	remaining: 3.13s
100:	learn: 0.2805259	total: 351ms	remaining: 3.13s
101:	learn: 0.2800143	total: 354ms	remaining: 3.11s
102:	learn: 0.2790734	total: 357ms	remaining: 3.11s
103:	learn: 0.2784195	total:

263:	learn: 0.1919440	total: 875ms	remaining: 2.44s
264:	learn: 0.1913830	total: 878ms	remaining: 2.43s
265:	learn: 0.1911765	total: 881ms	remaining: 2.43s
266:	learn: 0.1910227	total: 884ms	remaining: 2.43s
267:	learn: 0.1905985	total: 888ms	remaining: 2.42s
268:	learn: 0.1904778	total: 891ms	remaining: 2.42s
269:	learn: 0.1903896	total: 895ms	remaining: 2.42s
270:	learn: 0.1902135	total: 898ms	remaining: 2.42s
271:	learn: 0.1900024	total: 901ms	remaining: 2.41s
272:	learn: 0.1896979	total: 904ms	remaining: 2.41s
273:	learn: 0.1895013	total: 907ms	remaining: 2.4s
274:	learn: 0.1891983	total: 911ms	remaining: 2.4s
275:	learn: 0.1891174	total: 914ms	remaining: 2.4s
276:	learn: 0.1889669	total: 917ms	remaining: 2.39s
277:	learn: 0.1886057	total: 920ms	remaining: 2.39s
278:	learn: 0.1882172	total: 923ms	remaining: 2.38s
279:	learn: 0.1876937	total: 926ms	remaining: 2.38s
280:	learn: 0.1872939	total: 929ms	remaining: 2.38s
281:	learn: 0.1872059	total: 933ms	remaining: 2.37s
282:	learn: 0.1

463:	learn: 0.1419145	total: 1.45s	remaining: 1.68s
464:	learn: 0.1416517	total: 1.46s	remaining: 1.68s
465:	learn: 0.1413704	total: 1.46s	remaining: 1.67s
466:	learn: 0.1411931	total: 1.46s	remaining: 1.67s
467:	learn: 0.1410294	total: 1.47s	remaining: 1.67s
468:	learn: 0.1408154	total: 1.47s	remaining: 1.67s
469:	learn: 0.1407581	total: 1.47s	remaining: 1.66s
470:	learn: 0.1407329	total: 1.48s	remaining: 1.66s
471:	learn: 0.1404963	total: 1.48s	remaining: 1.66s
472:	learn: 0.1400978	total: 1.48s	remaining: 1.65s
473:	learn: 0.1398484	total: 1.48s	remaining: 1.65s
474:	learn: 0.1396575	total: 1.49s	remaining: 1.64s
475:	learn: 0.1395079	total: 1.49s	remaining: 1.64s
476:	learn: 0.1394211	total: 1.49s	remaining: 1.64s
477:	learn: 0.1392149	total: 1.5s	remaining: 1.64s
478:	learn: 0.1390722	total: 1.5s	remaining: 1.63s
479:	learn: 0.1388479	total: 1.5s	remaining: 1.63s
480:	learn: 0.1386221	total: 1.51s	remaining: 1.63s
481:	learn: 0.1384265	total: 1.51s	remaining: 1.62s
482:	learn: 0.1

645:	learn: 0.1122466	total: 2.04s	remaining: 1.12s
646:	learn: 0.1121462	total: 2.04s	remaining: 1.11s
647:	learn: 0.1119211	total: 2.05s	remaining: 1.11s
648:	learn: 0.1117178	total: 2.05s	remaining: 1.11s
649:	learn: 0.1116795	total: 2.06s	remaining: 1.11s
650:	learn: 0.1115214	total: 2.06s	remaining: 1.1s
651:	learn: 0.1113641	total: 2.06s	remaining: 1.1s
652:	learn: 0.1113299	total: 2.06s	remaining: 1.1s
653:	learn: 0.1112645	total: 2.07s	remaining: 1.09s
654:	learn: 0.1111459	total: 2.07s	remaining: 1.09s
655:	learn: 0.1109755	total: 2.07s	remaining: 1.09s
656:	learn: 0.1108548	total: 2.08s	remaining: 1.08s
657:	learn: 0.1107417	total: 2.08s	remaining: 1.08s
658:	learn: 0.1105636	total: 2.08s	remaining: 1.08s
659:	learn: 0.1104205	total: 2.09s	remaining: 1.07s
660:	learn: 0.1103401	total: 2.09s	remaining: 1.07s
661:	learn: 0.1101157	total: 2.1s	remaining: 1.07s
662:	learn: 0.1099054	total: 2.1s	remaining: 1.07s
663:	learn: 0.1098534	total: 2.1s	remaining: 1.06s
664:	learn: 0.1098

836:	learn: 0.0901724	total: 2.63s	remaining: 511ms
837:	learn: 0.0900194	total: 2.63s	remaining: 508ms
838:	learn: 0.0899148	total: 2.63s	remaining: 505ms
839:	learn: 0.0898151	total: 2.64s	remaining: 502ms
840:	learn: 0.0897179	total: 2.64s	remaining: 499ms
841:	learn: 0.0896593	total: 2.64s	remaining: 496ms
842:	learn: 0.0895647	total: 2.65s	remaining: 493ms
843:	learn: 0.0894621	total: 2.65s	remaining: 490ms
844:	learn: 0.0893290	total: 2.65s	remaining: 487ms
845:	learn: 0.0891843	total: 2.65s	remaining: 483ms
846:	learn: 0.0891589	total: 2.66s	remaining: 480ms
847:	learn: 0.0890776	total: 2.66s	remaining: 477ms
848:	learn: 0.0890376	total: 2.66s	remaining: 474ms
849:	learn: 0.0888726	total: 2.67s	remaining: 471ms
850:	learn: 0.0887236	total: 2.67s	remaining: 467ms
851:	learn: 0.0886365	total: 2.67s	remaining: 464ms
852:	learn: 0.0884630	total: 2.67s	remaining: 461ms
853:	learn: 0.0883187	total: 2.68s	remaining: 458ms
854:	learn: 0.0882639	total: 2.68s	remaining: 454ms
855:	learn: 

In [135]:
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 0.891 (0.025)


In [131]:
results=best_grid.predict(x_test)
results=sc_sp.inverse_transform(results)

In [132]:
result_data = pd.read_csv("sample_submission.csv")
result_data

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977
...,...,...
1454,2915,167081.220949
1455,2916,164788.778231
1456,2917,219222.423400
1457,2918,184924.279659


In [133]:
output = pd.DataFrame({'Id': result_data.Id, 'SalePrice': results})
output.to_csv('my_submission.csv', index=False)